In [98]:
import numpy as np
import pandas as pd
import datetime
import urllib
import sqlite3

In [137]:
## Read in our data. We've aggregated our data by date and limited the scope to BROOKLYN for this example.
query =  ("https://data.cityofnewyork.us/resource/qiz3-axqb.json?"
          "$where=borough=%22BROOKLYN%22"
          "&$limit=5000"
          "&$order=date%20DESC")
raw_data = pd.read_json(query)

In [187]:
raw_data = pd.DataFrame()
offset = 0
limit = 50000

query =  ("https://data.cityofnewyork.us/resource/qiz3-axqb.json?" + 
          "$where=borough=%22BROOKLYN%22" +
          "&$order=date%20DESC" +
          "&$limit=" + str(limit) + 
          "&$offset=" + str(offset))

while(not pd.read_json(query).empty):
    data = pd.read_json(query)
    print("Obtaining Data from ") + str(offset) + " to " + str(offset + limit)
    raw_data = raw_data.append(data)
    offset = offset + 50000
    query =  ("https://data.cityofnewyork.us/resource/qiz3-axqb.json?" + 
              "$where=borough=%22BROOKLYN%22" +
              "&$order=date%20DESC" +
              "&$limit=" + str(limit) + 
              "&$offset=" + str(offset))

## Drop location column
raw_data = raw_data.drop('location', axis=1)

Obtaining Data from 0 to 50000
Obtaining Data from 50000 to 100000
Obtaining Data from 100000 to 150000
Obtaining Data from 150000 to 200000
Obtaining Data from 200000 to 250000


In [217]:
raw_data.columns

Index([u'borough', u'contributing_factor_vehicle_1',
       u'contributing_factor_vehicle_2', u'contributing_factor_vehicle_3',
       u'contributing_factor_vehicle_4', u'contributing_factor_vehicle_5',
       u'cross_street_name', u'date', u'latitude', u'longitude',
       u'number_of_cyclist_injured', u'number_of_cyclist_killed',
       u'number_of_motorist_injured', u'number_of_motorist_killed',
       u'number_of_pedestrians_injured', u'number_of_pedestrians_killed',
       u'number_of_persons_injured', u'number_of_persons_killed',
       u'off_street_name', u'on_street_name', u'time', u'unique_key',
       u'vehicle_type_code1', u'vehicle_type_code2', u'vehicle_type_code_3',
       u'vehicle_type_code_4', u'vehicle_type_code_5', u'zip_code'],
      dtype='object')

In [207]:
## Create a connection to our database.
conn = sqlite3.connect("vehicle.db")

In [214]:
## Create a cursor object and verify that we have an empty table called "collisions"
cur = conn.cursor()
cur.execute("select date, count(*) from collisions group by date;")
results = cur.fetchall()
print results

[(u'2012-07-01 00:00:00', 135), (u'2012-07-02 00:00:00', 140), (u'2012-07-03 00:00:00', 167), (u'2012-07-04 00:00:00', 94), (u'2012-07-05 00:00:00', 153), (u'2012-07-06 00:00:00', 131), (u'2012-07-07 00:00:00', 111), (u'2012-07-08 00:00:00', 112), (u'2012-07-09 00:00:00', 117), (u'2012-07-10 00:00:00', 128), (u'2012-07-11 00:00:00', 140), (u'2012-07-12 00:00:00', 147), (u'2012-07-13 00:00:00', 143), (u'2012-07-14 00:00:00', 123), (u'2012-07-15 00:00:00', 114), (u'2012-07-16 00:00:00', 138), (u'2012-07-17 00:00:00', 145), (u'2012-07-18 00:00:00', 114), (u'2012-07-19 00:00:00', 104), (u'2012-07-20 00:00:00', 122), (u'2012-07-21 00:00:00', 95), (u'2012-07-22 00:00:00', 131), (u'2012-07-23 00:00:00', 130), (u'2012-07-24 00:00:00', 124), (u'2012-07-25 00:00:00', 136), (u'2012-07-26 00:00:00', 128), (u'2012-07-27 00:00:00', 140), (u'2012-07-28 00:00:00', 110), (u'2012-07-29 00:00:00', 62), (u'2012-07-30 00:00:00', 131), (u'2012-07-31 00:00:00', 119), (u'2012-08-01 00:00:00', 113), (u'2012-08

In [215]:
results[0]

(u'2012-07-01 00:00:00', 135)

In [202]:
## Push the dataframe into the collisions table.
raw_data.to_sql("collisions", conn, if_exists="replace")

In [206]:
## Confirm that we have 5000 rows in the table
cur.execute("select count(*) from collisions")
results = cur.fetchall()
print results

[(234141,)]
